<a href="https://colab.research.google.com/github/ramahasiba/NLP/blob/LangChain/Build_a_Simple_LLM_Application_With_Chat_Models_and_Prompt_Templates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Build a Simple LLM Application with Chat Models and Prompt Templates](https://python.langchain.com/docs/tutorials/llm_chain/)

## Installation

In [1]:
!pip install python-dotenv -q
!pip install transformers -q
!pip install langchain -q

In [3]:
!pip install -qU "langchain[groq]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 3.8 MB/s eta 0:00:00


## Setup

In [2]:
import os
from pprint import pprint
from dotenv import load_dotenv
import getpass

try:
  load_dotenv('.env')
except ImportError:
  print('No .env file found')

# Setup LangSmith to be able to inspect what exactly goes inside my chain or agent
os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
  os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
      prompt = "Enter the Langsmith api key:"
  )

if "LANGSMITH_PROJECT" not in os.environ:
  os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
      prompt = "Enter langsmith project name: "
  )
  if not os.environ.get("LANGSMITH_PROJECT"):
    os.environ["LANGSMITH_PROJECT"] = "default"

os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')
os.environ["HF_TOKEN"] = os.getenv('HF_TOKEN')

Enter the Langsmith api key:··········
Enter langsmith project name: ··········


## Using LLM

In [4]:
from langchain.chat_models import init_chat_model # Chat model is unstance of the runnable interface
model_name = "llama3-70b-8192"

model=init_chat_model(model_name, model_provider="groq")

In [13]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the follwoing from English to French"),
    HumanMessage("Hi, how are you?"),
]

response = model.invoke(messages)

the invoke method takes inputs and generate output of type message objects. it accepts inputs of type String and the openAI format.

In [7]:
type(response)

langchain_core.messages.ai.AIMessage

In [8]:
model.invoke("Hello, what is your name?")

AIMessage(content='Nice to meet you! I don\'t have a personal name, but I\'m an AI designed to assist and communicate with users in a helpful and friendly way. You can call me "Assistant" or "AI" if you like! What brings you here today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 17, 'total_tokens': 71, 'completion_time': 0.154285714, 'prompt_time': 0.000506862, 'queue_time': 0.054027948000000006, 'total_time': 0.154792576}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--8c766189-9846-4a3f-b233-1c8fba6821f7-0', usage_metadata={'input_tokens': 17, 'output_tokens': 54, 'total_tokens': 71})

In [9]:
model.invoke([
    {"role": "user", "content": "Hello"}
])

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 11, 'total_tokens': 37, 'completion_time': 0.074285714, 'prompt_time': 0.000129829, 'queue_time': 0.054252281, 'total_time': 0.074415543}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--cdae7aa3-2b66-47a4-8d06-abcf9494704f-0', usage_metadata={'input_tokens': 11, 'output_tokens': 26, 'total_tokens': 37})

In [11]:
model.invoke([HumanMessage("Hello")])

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 11, 'total_tokens': 37, 'completion_time': 0.074285714, 'prompt_time': 0.000243578, 'queue_time': 0.054642602, 'total_time': 0.074529292}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--eeaa8424-1536-45ee-83fc-779db874f52a-0', usage_metadata={'input_tokens': 11, 'output_tokens': 26, 'total_tokens': 37})

## Streaming
streaming is a mode of invokation that Runnnable models provides. This allows us to stream individual tokens from a chat model.

In [23]:
for token in model.stream(messages):
  print(token.content, end=" ")

 Bonjour ,  comment  vas -t u ?  

## Prompt Templates
The application usually contains logic and to pass and combine user input according to the logic, we format a template with user input.

**Prompt Templates** are a concept inLangchain that is designed to assist with transforming user input into the LLM application logic.

In [16]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [17]:
prompt = prompt_template.invoke({
    "language": "French",
    "text": "hi"
})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into French', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})])

In [20]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

In [21]:
response = model.invoke(prompt)
print(response.content)

Salut !
